In [29]:
from configparser import ConfigParser
import psycopg2.extras
import pandas as pd

In [31]:
#sign into database


In [33]:
with conn.cursor() as cursor:
        print('PostgreSQL database version:')
        cursor.execute('SELECT version()')
        # display the PostgreSQL database server version
        db_version = cursor.fetchone()
        cursor.execute ("select id, created_date as date from caltrans_cctv_archive order by date desc limit 100")
        list = cursor.fetchall()
        df = pd.DataFrame(list, columns = ["Id","Date"])

PostgreSQL database version:


In [35]:
for i in df

,Id,Date
0,633fac5d-37d7-11ef-aa87-0242ac110004,2024-07-01 18:26:09.446507+00:00
1,8cad82bd-37d6-11ef-aa87-0242ac110004,2024-07-01 18:20:09.461765+00:00
2,68d564e1-37d6-11ef-aa87-0242ac110004,2024-07-01 18:19:09.318514+00:00
3,b5cc7fa5-37d5-11ef-aa87-0242ac110004,2024-07-01 18:14:08.949388+00:00
4,b5caf7cc-37d5-11ef-aa87-0242ac110004,2024-07-01 18:14:08.942626+00:00
...,...,...
95,a584b042-37cd-11ef-ac6d-0242ac110004,2024-07-01 17:16:25.716821+00:00
96,a5839ab0-37cd-11ef-ac6d-0242ac110004,2024-07-01 17:16:25.653146+00:00
97,a57e8907-37cd-11ef-ac6d-0242ac110004,2024-07-01 17:16:25.641297+00:00
98,a57f7a7b-37cd-11ef-ac6d-0242ac110004,2024-07-01 17:16:25.640589+00:00


In [59]:
for index,row in df.iterrows():
    test_string = "https://api.traffic.labrador.dev/caltrans_cctv_archive/" + row['Id']
    print(test_string)

https://api.traffic.labrador.dev/caltrans_cctv_archive/633fac5d-37d7-11ef-aa87-0242ac110004
https://api.traffic.labrador.dev/caltrans_cctv_archive/8cad82bd-37d6-11ef-aa87-0242ac110004
https://api.traffic.labrador.dev/caltrans_cctv_archive/68d564e1-37d6-11ef-aa87-0242ac110004
https://api.traffic.labrador.dev/caltrans_cctv_archive/b5cc7fa5-37d5-11ef-aa87-0242ac110004
https://api.traffic.labrador.dev/caltrans_cctv_archive/b5caf7cc-37d5-11ef-aa87-0242ac110004
https://api.traffic.labrador.dev/caltrans_cctv_archive/b5cae228-37d5-11ef-aa87-0242ac110004
https://api.traffic.labrador.dev/caltrans_cctv_archive/b5c7601d-37d5-11ef-aa87-0242ac110004
https://api.traffic.labrador.dev/caltrans_cctv_archive/6e8dd91b-37d5-11ef-aa87-0242ac110004
https://api.traffic.labrador.dev/caltrans_cctv_archive/6e8d5123-37d5-11ef-aa87-0242ac110004
https://api.traffic.labrador.dev/caltrans_cctv_archive/4accff28-37d5-11ef-aa87-0242ac110004
https://api.traffic.labrador.dev/caltrans_cctv_archive/4acdc030-37d5-11ef-aa87-0